In [1]:
using LinearAlgebra
using DataFrames,CSV
using Plots
using Statistics
using BetaML: partition
using Flux
using Distributions
using ProgressMeter


In [2]:
function fc(Rij::T, Rc::T) :: T where {T}
    #se sto oltre al cutoff restituisco 0
    if Rij>=Rc
        return 0
    else 
        arg=1-1/(1-(Rij/Rc)*(Rij/Rc))

        return exp(arg)

    end
end

fc (generic function with 1 method)

In [3]:
#non sto usando le unità di misura vere per semplificare la task

function LennardJones(r::T) :: T where {T}
    epsilon=1 #in unità di epsilon
    sigma=1 #in unità di sigma
    temp=sigma/r
    V=4*epsilon*(temp^12-temp^6)
    return V
end

LennardJones (generic function with 1 method)

In [4]:
# Funzione per calcolare la distanza tra due punti nello spazio 3D
function distanza(p1, p2)
    return norm(p1 - p2)
end

# Generare le coordinate rispettando il criterio della distanza minima
function genera_coordinate!(Data, Lato, distanza_minima)
    N_Data, N_Elio, _ = size(Data)
    
    for i in 1:N_Data
        for j in 1:N_Elio
            punto_valido = false
            nuovo_punto = zeros(3)  # Inizializzare nuovo_punto prima del ciclo while
            while !punto_valido
                nuovo_punto .= rand(3) .* Lato .- Lato / 2
                punto_valido = true
                for k in 1:(j-1)
                    if distanza(nuovo_punto, Data[i, k, 2:4]) < distanza_minima
                        punto_valido = false
                        break
                    end
                end
            end
            Data[i, j, 2:4] .= nuovo_punto
        end
    end
end

genera_coordinate! (generic function with 1 method)

In [5]:
function custom_loss(model, x, y)
    total_loss = 0.0  # Accumulatore per la perdita totale

    for k in 1:size(x)[1]
        temp = 0.0
        for i in 1:N_Elio
            temp += model(x[k, i, :])[1]
        end
        total_loss += abs2(temp - y[k])
    end

    # Restituisci la media della perdita
    return total_loss / size(x)[1]
end


custom_loss (generic function with 1 method)

In [6]:
#definisco il mio layer personalizzato 

struct MyLayer
    W_eta::AbstractArray  # Pesi per il collegamento "eta"
    W_Fs::AbstractArray   # Pesi per il collegamento "Fs"
    cutoff::Float32  # Raggio di cutoff, fisso, lo prende come argomento il layer

end

function MyLayer(input_dim::Int, hidden_dim::Int, cutoff::Float32)
    W_eta = Float32.(rand(Uniform(0.25, 2.5), hidden_dim, input_dim))  # Inizializza i pesi come Float32
    W_Fs = Float32.(rand(Uniform(0.0, 2.5), hidden_dim, input_dim))    # Inizializza i pesi come Float32
    MyLayer(W_eta, W_Fs, cutoff)
end

function (layer::MyLayer)(x)
    N_Atomi_Vicini=size(x)[1]
    sum=zeros(size(layer.W_eta)[1])

    for j in 1:N_Atomi_Vicini
        sum=sum.+fc(x[j],layer.cutoff).*exp.(-(x[j].-layer.W_Fs).*(x[j].-layer.W_Fs).*layer.W_eta)
    end
    return Float32.(sum)
end

function Base.show(io::IO, layer::MyLayer)
    println(io, "CustomLayer with two links per input neuron (eta and Fs)")
    println(io, "Weights eta: ", layer.W_eta)
    println(io, "Weights Fs: ", layer.W_Fs)
end

Flux.@layer MyLayer

In [7]:
#da qui Genero il dataset

# Definire le dimensioni della scatola e il numero di dati
R_cutoff = 2.5f0  #raggio della sfera di cutoff
Lato = 4
N_Data = 1000 # Numero di configurazioni (dati)
N_Elio = 15# Numero di atomi di elio nella scatola
distanza_minima = 0.95f0

# Inizializzare il dataset
Data = zeros(Float32,N_Data, N_Elio,4)
Data[:, :, 1] .= 2  # Metto la carica dei nuclei nel primo slot

# Genero le coordinate
genera_coordinate!(Data, Lato, distanza_minima)

In [8]:
# Inizializzo un array per memorizzare l'energia totale di ogni dataset
EnergieTotali = zeros(Float32, N_Data)

# Inizializzo un array vuoto per memorizzare le distanze di ogni dataset
array_Rij = zeros(Float32, N_Data, N_Elio, N_Elio-1)

# Loop attraverso tutte le configurazioni di dati
for i in 1:N_Data
    total_energy_dataset = 0.0  # Energia totale per il dataset i
    
    for j in 1:N_Elio
        idx = 1  # Indice per memorizzare le distanze evitando l'atomo stesso
        
        for k in 1:N_Elio
            if j != k
                # Calcolo delle distanze tra j e k
                dx = Data[i, j, 2] - Data[i, k, 2]
                dy = Data[i, j, 3] - Data[i, k, 3]
                dz = Data[i, j, 4] - Data[i, k, 4]
                r = sqrt(dx^2 + dy^2 + dz^2)
                
                # Memorizza la distanza nell'array
                array_Rij[i, j, idx] = r
                idx += 1
                
                # Somma l'energia di interazione tra atomi j e k
                total_energy_dataset += LennardJones(r)
            end
        end
    end
    
    # Memorizza l'energia totale del dataset i
    EnergieTotali[i] = total_energy_dataset/2
end


In [9]:
# Partitioning
((x_train, tempX), (y_train, tempY)) = partition([array_Rij, EnergieTotali], [0.7, 0.3])
((x_vali, x_test), (y_vali, y_test)) = partition([tempX, tempY], [0.5, 0.5])

# Permutazione delle dimensioni
x_train = permutedims(x_train, (1, 2, 3))
x_test = permutedims(x_test, (1, 2, 3))
x_vali = permutedims(x_vali, (1, 2, 3))

y_train = permutedims(y_train)
y_test = permutedims(y_test)
y_vali = permutedims(y_vali)

# Assicurati che siano tutti Float32
x_train = Float32.(x_train)
x_test = Float32.(x_test)
x_vali = Float32.(x_vali)

y_train = Float32.(y_train)
y_test = Float32.(y_test)
y_vali = Float32.(y_vali)



1×150 Matrix{Float32}:
 -8.50928  -11.2122  -7.95823  -6.90699  …  -7.32692  -7.30311  -6.6809

In [10]:
# Definizione del modello
G1_Number = 5  # Numero di funzioni G1

# Modello per un singolo atomo
# questo prende le distanze tra un atomo e tutti gli altri atomi del sistema 
# e calcola la sua energia ( anche se non è ben definita )
single_atom_model = Chain(
    MyLayer(1, G1_Number, R_cutoff),
    Dense(G1_Number, 10, tanh),
    Dense(10, 5, tanh),
    Dense(5, 1)
)

Base.show(MyLayer(1, G1_Number, R_cutoff))

CustomLayer with two links per input neuron (eta and Fs)
Weights eta: Float32[1.8717332; 0.7723369; 0.6382894; 1.735458; 0.49239781;;]
Weights Fs: Float32[0.65873367; 1.8493968; 2.183698; 2.0877883; 0.16286048;;]


In [11]:
# Inizializza i parametri per il learning rate adattivo
initial_lr = 0.01
min_lr = 1e-5
lr = initial_lr  # Learning rate iniziale
patience = 100  # Numero di epoche senza miglioramento per ridurre il learning rate
decay_factor = 0.5  # Fattore di riduzione del learning rate
no_improve_count = 0  # Contatore per le epoche senza miglioramento

# Configura l'ottimizzatore
opt = Flux.setup(Flux.Adam(lr), single_atom_model)

epochs = 2500
best_epoch = 0
best_loss = Inf
best_model = deepcopy(single_atom_model)  # Per salvare il miglior modello

LossTrain = zeros(Float32, epochs)
LossVali = zeros(Float32, epochs)

println(" ")


In [ ]:
# Inizializza la barra di progresso
@showprogress for epoch in 1:epochs
    # Training in batch
    batch_size = 32  # Dimensione del batch
    for i in 1:batch_size:size(x_train, 1)
        # Limita l'indice per non superare le dimensioni dell'array
        end_index = min(i + batch_size - 1, size(x_train, 1))
        
        # Crea il mini-batch
        x_batch = x_train[i:end_index, :, :]
        y_batch = y_train[i:end_index]

        # Aggiorna il modello
        Flux.train!(custom_loss, single_atom_model, [(x_batch, y_batch)], opt)
    end

    # Calcolo delle perdite
    LossTrain[epoch] = custom_loss(single_atom_model, x_train, y_train)
    LossVali[epoch] = custom_loss(single_atom_model, x_vali, y_vali)

    # Salva il miglior modello basato sulla perdita di validazione
    if LossVali[epoch] < best_loss
        best_epoch = epoch
        best_loss = LossVali[epoch]
        best_model = deepcopy(single_atom_model)
        no_improve_count = 0  # Reset del contatore
    else
        no_improve_count += 1
    end

    # Riduci il learning rate se non ci sono stati miglioramenti per un numero di epoche pari a 'patience'
    if no_improve_count >= patience
        lr = max(lr * decay_factor, min_lr)  # Riduci il learning rate ma non scendere sotto min_lr
        opt = Flux.setup(Flux.Adam(lr), single_atom_model)  # Reimposta l'ottimizzatore con il nuovo learning rate
        no_improve_count = 0  # Reset della conta
    end
end

# Sovrascrivi il modello attuale con il miglior modello trovato
single_atom_model = best_model

println("Final Training Loss: $(custom_loss(single_atom_model, x_train, y_train)) oppure $(sqrt(custom_loss(single_atom_model, x_train, y_train))*8.52e-1 ) eV")
println("Final Validation Loss: $(custom_loss(single_atom_model, x_vali, y_vali)) oppure $(sqrt(custom_loss(single_atom_model, x_vali, y_vali))*8.52e-1 ) eV")
println("Il modello migliore è stato trovato all'epoca: $best_epoch")


Progress:   2%|██                                       |  ETA: 2:43:59

In [ ]:
# Carica il file BSON
#data = BSON.load("best_model_no_relu.bson")

# Estrai il modello salvato
#single_atom_model = data[:single_atom_model]




In [ ]:
LossTrain_real=sqrt.(LossTrain).* 8.52e-1 #converto in eV
LossVali_real=sqrt.(LossVali).* 8.52e-1

plot(1:epochs, LossTrain_real, label="Train", yscale=:log10, ylabel="radice quadrata della Loss [meV]", xlabel="Epoca",
     title="Radice quadrata della Loss in funzione dell'epoca", color="blue", linestyle=:solid)
plot!(1:epochs, LossVali_real, label="Validation", color="red", linestyle=:dot)

# Linea per la migliore epoca≥
plot!([best_epoch, best_epoch], [minimum(LossTrain), 1], label="Best Epoch", color="black", linestyle=:dash, linewidth=2)

# Salva la figura
savefig("2E_BP_loss.png")



In [ ]:
# Array per salvare l'energia totale predetta riconvertita
predicted_energy_total = zeros(size(y_train))

# Calcola l'energia totale predetta per ciascun sistema, riportando i dati alla scala originale
for k in 1:size(x_train)[1]
    total_energy = 0.0
    for i in 1:N_Elio
        total_energy += single_atom_model(x_train[k, i, :])[1]
    end
    predicted_energy_total[k] = total_energy
end

# Calcolo del coefficiente di Pearson
pearson_coeff = cor(y_train', predicted_energy_total')[1]

# Limiti per la linea ideale basati sui dati originali
min_energy = minimum(y_train) * 1.1
max_energy = maximum(y_train) * 1.1

# Plot Predicted vs Real
plot([min_energy, max_energy], [min_energy, max_energy], color="black", label="Andamento ideale", legendfont=font(12))  # Modifica qui per aumentare la dimensione della legenda
scatter!(y_train', predicted_energy_total', label="Energia", alpha=0.5, color="red")

# Settaggi degli assi e del titolo
ylabel!("Valore predetto (ε)")  # Modificato qui
xlabel!("Valore reale (ε)")  # Modificato qui
title!("Train set: Valore predetto vs valore reale")
xlims!(min_energy, max_energy)
ylims!(min_energy, max_energy)

# Posizione dell'annotazione in termini percentuali
x_position = min_energy + 0.015 * (max_energy - min_energy)  # 97% lungo l'asse x
y_position = min_energy + 0.75 * (max_energy - min_energy)  # 55% lungo l'asse y

# Aggiungi il coefficiente di Pearson al plot
annotate!([(x_position, y_position, text("Coefficiente di Pearson: $(round(pearson_coeff, digits=3))", :left, 12))])

# Salva il plot
savefig("correlazione_train_15_atomi.png")


In [ ]:
# Array per salvare l'energia totale predetta riconvertita
predicted_energy_total_vali = zeros(size(y_vali))

# Calcola l'energia totale predetta per ciascun sistema nel validation set, riportando i dati alla scala originale
for k in 1:size(x_vali)[1]
    total_energy = 0.0
    for i in 1:N_Elio
        total_energy += single_atom_model(x_vali[k, i, :])[1]
    end
    predicted_energy_total_vali[k] = total_energy
end

# Calcolo del coefficiente di Pearson
pearson_coeff_vali = cor(y_vali', predicted_energy_total_vali')[1]

# Limiti per la linea ideale basati sui dati originali
min_energy_vali = minimum(y_vali) * 1.1
max_energy_vali = maximum(y_vali) * 1.1

# Plot Predicted vs Real (Validation set)
plot([min_energy_vali, max_energy_vali], [min_energy_vali, max_energy_vali], color="black", label="Andamento ideale", legendfont=font(12))  # Modifica qui per aumentare la dimensione della legenda
scatter!(y_vali', predicted_energy_total_vali', label="Energia", alpha=0.5, color="green")

# Settaggi degli assi e del titolo
ylabel!("Valore predetto (ε)")  # Modificato qui
xlabel!("Valore reale (ε)")  # Modificato qui
title!("Validation Set: Valore predetto vs valore reale")
xlims!(min_energy_vali, max_energy_vali)
ylims!(min_energy_vali, max_energy_vali)

# Posizione dell'annotazione in termini percentuali
x_position = min_energy_vali + 0.015 * (max_energy_vali - min_energy_vali)  # 97% lungo l'asse x
y_position = min_energy_vali + 0.75 * (max_energy_vali - min_energy_vali)  # 55% lungo l'asse y

# Aggiungi il coefficiente di Pearson al plot
annotate!([(x_position, y_position, text("Coefficiente di Pearson: $(round(pearson_coeff_vali, digits=3))", :left, 12))])  # Dimensione del testo modificata

# Salva il plot
savefig("correlazione_validation_15_atomi.png")



In [ ]:


# Array per salvare l'energia totale predetta riconvertita
predicted_energy_total_test = zeros(size(y_test))

# Calcola l'energia totale predetta per ciascun sistema nel validation set, riportando i dati alla scala originale
for k in 1:size(x_test)[1]
    total_energy = 0.0
    for i in 1:N_Elio
        total_energy += single_atom_model(x_test[k, i, :])[1]
    end
    predicted_energy_total_test[k] = total_energy
end


# Calcolo del coefficiente di Pearson
pearson_coeff_test = cor(y_test', predicted_energy_total_test')[1]

# Limiti per la linea ideale basati sui dati originali
min_energy_test = minimum(y_test) * 1.1
max_energy_test = maximum(y_test) * 1.1

# Plot Predicted vs Real (Validation set)

plot([min_energy_test, max_energy_test], [min_energy_test, max_energy_test], color="black", label="Andamento ideale", legendfont=font(12))
scatter!(y_test', predicted_energy_total_test', label="Energia", alpha=0.5, color="blue")

# Posizione dell'annotazione in termini percentuali
x_position = min_energy_test + 0.015 * (max_energy_test - min_energy_test)  # 1% lungo l'asse x
y_position = min_energy_test + 0.75 * (max_energy_test - min_energy_test)  # 85% lungo l'asse y

# Annotazione con posizione calcolata
annotate!([(x_position, y_position, text("Coefficiente di Pearson: $(round(pearson_coeff_test, digits=3))", :left, 12))])

# Settaggi degli assi e del titolo
ylabel!("Valore predetto (ε)")
xlabel!("Valore reale (ε)")
title!("Test Set: Valore predetto vs valore reale")
xlims!(min_energy_test, max_energy_test)
ylims!(min_energy_test, max_energy_test)

# Salva il plot
savefig("correlazione_test_15_atomi.png")


In [ ]:
using Plots

# Array per salvare l'energia totale predetta riconvertita per il validation set
predicted_energy_total_vali = zeros(size(y_vali))

# Calcola l'energia totale predetta per ciascun sistema nel validation set
for k in 1:size(x_vali)[1]
    total_energy = 0.0
    for i in 1:N_Elio
        total_energy += single_atom_model(x_vali[k, i, :])[1]
    end
    predicted_energy_total_vali[k] = total_energy
end

# Calcolo del coefficiente di Pearson per il validation set
pearson_coeff_vali = cor(y_vali', predicted_energy_total_vali')[1]

# Limiti per la linea ideale basati sui dati originali
min_energy_vali = minimum(y_vali) * 1.1
max_energy_vali = maximum(y_vali) * 1.1

# Plot Predicted vs Real (Validation set)
p1 = plot([min_energy_vali, max_energy_vali], [min_energy_vali, max_energy_vali], color="black", label="Andamento ideale", legendfont=font(12))
scatter!(p1, y_vali', predicted_energy_total_vali', label="Energia", alpha=0.5, color="green")

# Settaggi degli assi e del titolo
ylabel!(p1, "Valore predetto (ε)")
xlabel!(p1, "Valore reale (ε)")
title!(p1, "Validation Set: Valore predetto vs valore reale")
xlims!(p1, min_energy_vali, max_energy_vali)
ylims!(p1, min_energy_vali, max_energy_vali)

# Posizione dell'annotazione
x_position = min_energy_vali + 0.015 * (max_energy_vali - min_energy_vali)
y_position = min_energy_vali + 0.75 * (max_energy_vali - min_energy_vali)
annotate!(p1, [(x_position, y_position, text("Coefficiente di Pearson: $(round(pearson_coeff_vali, digits=3))", :left, 12))])

# Array per salvare l'energia totale predetta riconvertita per il test set
predicted_energy_total_test = zeros(size(y_test))

# Calcola l'energia totale predetta per ciascun sistema nel test set
for k in 1:size(x_test)[1]
    total_energy = 0.0
    for i in 1:N_Elio
        total_energy += single_atom_model(x_test[k, i, :])[1]
    end
    predicted_energy_total_test[k] = total_energy
end

# Calcolo del coefficiente di Pearson per il test set
pearson_coeff_test = cor(y_test', predicted_energy_total_test')[1]

# Limiti per la linea ideale basati sui dati originali
min_energy_test = minimum(y_test) * 1.1
max_energy_test = maximum(y_test) * 1.1

# Plot Predicted vs Real (Test set)
p2 = plot([min_energy_test, max_energy_test], [min_energy_test, max_energy_test], color="black", label="Andamento ideale", legendfont=font(12))
scatter!(p2, y_test', predicted_energy_total_test', label="Energia", alpha=0.5, color="blue")

# Settaggi degli assi e del titolo
ylabel!(p2, "Valore predetto (ε)")
xlabel!(p2, "Valore reale (ε)")
title!(p2, "Test Set: Valore predetto vs valore reale")
xlims!(p2, min_energy_test, max_energy_test)
ylims!(p2, min_energy_test, max_energy_test)


annotate!(p2, [(x_position, y_position, text("Coefficiente di Pearson: $(round(pearson_coeff_test, digits=3))", :left, 12))])

# Array per salvare l'energia totale predetta riconvertita per il training set
predicted_energy_total = zeros(size(y_train))

# Calcola l'energia totale predetta per ciascun sistema nel training set
for k in 1:size(x_train)[1]
    total_energy = 0.0
    for i in 1:N_Elio
        total_energy += single_atom_model(x_train[k, i, :])[1]
    end
    predicted_energy_total[k] = total_energy
end

# Calcolo del coefficiente di Pearson per il training set
pearson_coeff = cor(y_train', predicted_energy_total')[1]

# Limiti per la linea ideale basati sui dati originali
min_energy = minimum(y_train) * 1.1
max_energy = maximum(y_train) * 1.1

# Plot Predicted vs Real (Training set)
p3 = plot([min_energy, max_energy], [min_energy, max_energy], color="black", label="Andamento ideale", legendfont=font(12))
scatter!(p3, y_train', predicted_energy_total', label="Energia", alpha=0.5, color="red")

# Settaggi degli assi e del titolo
ylabel!(p3, "Valore predetto (ε)")
xlabel!(p3, "Valore reale (ε)")
title!(p3, "Train Set: Valore predetto vs valore reale")
xlims!(p3, min_energy, max_energy)
ylims!(p3, min_energy, max_energy)

# Posizione dell'annotazione per il training set
x_position_train = min_energy + 0.015 * (max_energy - min_energy)
y_position_train = min_energy + 0.75 * (max_energy - min_energy)
annotate!(p3, [(x_position, y_position, text("Coefficiente di Pearson: $(round(pearson_coeff, digits=3))", :left, 12))])

# Combinazione dei plot in un'unica figura
plot(p3, p1, p2, layout = (3, 1), size = (800, 800))  # Modificato a (3, 1) per accomodare 3 plot

# Salva l'immagine con i tre plot
savefig("correlazione_all_set.png")


In [ ]:
#Qui genero un sistema con due soli atomi di Elio e vedo come performa il modello allenato su 15 atomi
# Definizione dei parametri
Lato_2 = 2  # Dimensione del box
N_Data_2 = 100  # Numero di configurazioni (distanze)
N_Elio_2 = 2  # Numero di atomi di elio nel sistema
distanze_array = range(0.95, 2.7, length=N_Data_2)  # Distanze ordinate da testare

# Inizializzare dataset con due atomi per configurazione
Data_2 = zeros(Float32, N_Data_2, N_Elio_2, 4)
Data_2[:, :, 1] .= 2  # Carica nucleare nel primo slot

# Genera le configurazioni con distanze specifiche lungo un solo asse
for i in 1:N_Data_2
    dist = distanze_array[i]
    Data_2[i, 1, 2] = 0.0  # Primo atomo alle coordinate (0,0,0)
    Data_2[i, 2, 2] = dist  # Secondo atomo alla distanza lungo l'asse x
end

# Calcola l'energia totale e le distanze per ogni configurazione
EnergieTotali_2 = zeros(Float32, N_Data_2)
array_Rij_2 = zeros(Float32, N_Data_2, N_Elio_2, N_Elio_2 - 1)

for i in 1:N_Data_2
    dx = Data_2[i, 1, 2] - Data_2[i, 2, 2]
    r = abs(dx)
    
    # Memorizza la distanza
    array_Rij_2[i, 1, 1] = r
    array_Rij_2[i, 2, 1] = r
    
    # Calcola l'energia di interazione usando il potenziale di Lennard-Jones
    EnergieTotali_2[i] = LennardJones(r)
end

# Calcola le predizioni di energia totale
predicted_energy_total_2 = zeros(Float32, N_Data_2)

for i in 1:N_Data_2
    total_energy = 0.0
    for j in 1:N_Elio_2
        total_energy += single_atom_model(array_Rij_2[i, j, :])[1]
    end
    predicted_energy_total_2[i] = total_energy / 2
end

# Plot dei risultati
x = range(0.95, 2.7, length=100)
y = LennardJones.(x)

plot(x, y, label="Potenziale di Lennard-Jones reale", color="black", lw=2)
plot!(distanze_array, predicted_energy_total_2, lw=2, color="red", label="Predizioni modello", alpha=0.6)
xlabel!("Distanza tra gli atomi (σ)")
ylabel!("Energia totale (ε)")
title!("Confronto tra potenziale reale e potenziale ML")

savefig("Predict_LJ_modello_NE.png")

min_finder=zeros(Float32, 2000,2,1)
energies=zeros(Float32, 2000)
for i in 1:2000
    min_finder[i,1,1]=1.09+i*0.001
    min_finder[i,2,1]=1.09+i*0.001
    energies[i]=single_atom_model(min_finder[i,:,:])[1]* 8.52e-1
end

println("il minimo si trova per r=$(min_finder[argmin(energies),1,1])")
println("il minimo vale E_min=$(minimum(energies))")

In [ ]:
using Plots

# Calcola la differenza tra il potenziale predetto e quello reale
differenza = abs.(predicted_energy_total_2 .- y)

# Grafico del potenziale reale e delle predizioni del modello
plot(
    x, y,
    label="Potenziale di Lennard-Jones reale",
    color="black",
    lw=2,
    xlabel="Distanza tra gli atomi (σ)",
    ylabel="Energia totale (ε)",
    title="Confronto tra potenziale reale e predetto"
)
plot!(
    distanze_array, predicted_energy_total_2,
    lw=2,
    color="red",
    label="Predizioni modello",
    alpha=0.6
)

# Aggiunta di un secondo asse verticale a destra per la differenza
plot!(
    twinx(),
    distanze_array, differenza,
    label="Differenza (Predetto - Reale)",
    color="blue",
    lw=2,
    linestyle=:dash,
    ylabel="Differenza (ε)"
)

# Salva il grafico
savefig("Predict_LJ_con_differenza.png")
